# Лабораторная работа 2. Элементы линейной алгебры. Шаргин Иван

In [2]:
import numpy as np
import matplotlib.pyplot as plt

Решим СЛАУ A * x = f 

## 1. Метод Гаусса

Функция Gauss_solve реализует метод Гаусса, принимает на вход матрицу A, вектор правой части f, возвращает решение u

In [3]:
def Gauss(A, f):
    """Функция Gauss реализует метод Гаусса, 
    принимает на вход матрицу A, вектор правой части f, 
    возвращает решение СЛАУ"""
    leading_element_inds = np.transpose(np.nonzero(np.abs(A) == np.abs(A).max()))[0]
    A_f = np.hstack((A, f))
    A_f[[0, leading_element_inds[0]]] = A_f[[leading_element_inds[0], 0]]
    A_f[:, [0, leading_element_inds[1]]] = A_f[:, [leading_element_inds[1], 0]]

    for j in range(0, A_f.shape[0] - 1):
        for i in range(j + 1, A_f.shape[0]):
            k = A_f[i, j] / A_f[j, j]
            A_f[i, j:] = A_f[i, j:] - A_f[j, j:] * k
    
            
    for j in range(A_f.shape[0] - 1, 0, -1):
        for i in range(j - 1, -1, -1):
            k = A_f[i, j] / A_f[j, j]
            A_f[i, :] = A_f[i, :] - A_f[j, :] * k
    
            
    for i in range(A_f.shape[0]):
        A_f[i, :] = A_f[i, :] / A_f[i, i]
    
 
    answer = A_f[:, -1]
    answer[[0, leading_element_inds[1]]] = answer[[leading_element_inds[1], 0]]
    
    
    return answer  


In [4]:
A_test = np.array([[5.0, 7.0, 6.0], [4.0, 8.0, 3.0],  [-1.0, 6.0, 2.0]])
print('A:')
print(A_test)
f_test = np.array([[35.0], [25.0], [10.0]])
print('f:')
print(f_test)
print('The answer:')
print(Gauss(A_test, f_test))

A:
[[ 5.  7.  6.]
 [ 4.  8.  3.]
 [-1.  6.  2.]]
f:
[[35.]
 [25.]
 [10.]]
The answer:
[2. 1. 3.]


Точное решение (2, 1, 3), что совпадает с полученным методом Гаусса результатом

In [5]:
A_c = np.array([[2., -1., 0], [5., 4., 2.], [0, 1., -3.]])
f_c = np.array([[3.], [6.], [2.]])
print(Gauss(A_c, f_c))

[ 1.48837209 -0.02325581 -0.6744186 ]


## 1.b Метод прогонки

In [6]:
def diags_3(A, f):
    """Функция реализует метод прогонки СЛАУ.
    Принимает матрицу А левой части и вектор f правой части, 
    необходимую точность. Возвращает вектор решения"""
    N = A.shape[0]
    d = f
    c = np.diagonal(A, offset=1)
    b = np.diagonal(A)
    a = np.diagonal(A, offset=-1)
    
    L = np.tril(A, k=-2)
    U = np.triu(A, k=2)
    if np.linalg.norm(L) > 0 or np.linalg.norm(U) > 0:
        print("Not 3x diagonal matrix!")
        return(0)
    
    answer = [0] * N
    y = []
    y.append(b[0])
    alpha = []
    alpha.append(-c[0] / y[0])
    beta = []
    beta.append(d[0]/y[0])
    
    for i in range(1, N-1):
        y.append(b[i] + a[i-1] * alpha[-1])
        alpha.append(-c[i] / y[-1])
        beta.append((d[i] - a[i-1] * beta[-1]) / y[-1])
    y.append(b[N-1] + a[N-2] * alpha[-1])
    beta.append((d[N-1] - a[N-2] * beta[-1]) / y[-1])
    
    answer[-1] = beta[-1]
    for i in range(N-2, -1, -1):
        answer[i] = alpha[i] * answer[i+1] + beta[i]
    answer = np.array(answer)

    print("answer:")
    print(answer)
    return(0)  

In [7]:
print(A_c)
print(f_c)
diags_3(A_c, f_c)

[[ 2. -1.  0.]
 [ 5.  4.  2.]
 [ 0.  1. -3.]]
[[3.]
 [6.]
 [2.]]
answer:
[[ 1.48837209]
 [-0.02325581]
 [-0.6744186 ]]


0

In [8]:
A_d = np.array([[1., 1., 0.], [1., 2., 1.], [0., 1., -1.]])
f_d = np.array([[3.], [8.], [-1.]])
print(A_d)
print(f_d)
diags_3(A_d, f_d)

[[ 1.  1.  0.]
 [ 1.  2.  1.]
 [ 0.  1. -1.]]
[[ 3.]
 [ 8.]
 [-1.]]
answer:
[[1.]
 [2.]
 [3.]]


0

In [9]:
diags_3(A_test, f_test)

Not 3x diagonal matrix!


0

## 2a. Метод Якоби решения СЛАУ

In [10]:
print(np.eye(3))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [11]:
def Jacobi(A, f, e):
    """Функция реализует метод Якоби решения СЛАУ.
    Принимает матрицу А левой части и вектор f правой части, 
    необходимую точность e. Возвращает вектор решения"""
    N = A.shape[0]
    diagonal = A.diagonal()
    
    L = np.tril(A, k=-1)
    U = np.triu(A, k=1)
    D = np.diag(np.diag(A))
    
    D_inv = np.linalg.inv(D)
    B = np.eye(N) - D_inv.dot(A)
    norm_B = np.linalg.norm(B, ord=np.inf)
    if norm_B >= 1:
        print("Can't use Jacobi for this system")
        return(-1)
    
    x = []
    x.append(f)
    x_new = f
    error = 100    
    
    while error >= e:
        a_x = A.dot(x[-1])
        x.append(np.array([(f[i] - a_x[i] + x[-1] [i] * diagonal[i]) / diagonal[i] for i in range(N)]))
        
        error = np.linalg.norm(x[-2] - x[-1] ) 
    
    print("answer:")
    print(x[-1])
    return(0)  

In [12]:
# Еще одна тестовая СЛАУ A_j * x = f_j, ее точное решение (1, 1)
A_j = np.array([[5., 2.], [1., 5.]])
f_j = np.array([[7.], [6.]])
print('A:')
print(A_j)
print('f:')
print(f_j)

A:
[[5. 2.]
 [1. 5.]]
f:
[[7.]
 [6.]]


In [13]:
Jacobi(A_j, f_j, 0.001)

answer:
[[0.99991808]
 [0.99995085]]


0

In [14]:
Jacobi(A_test, f_test, 0.001)

Can't use Jacobi for this system


-1

Точное решение (1, 1), что хорошо сходится с полученным по методу Якоби.

## 2b. Метод Зейделя.

In [15]:
def Zeidel(A, f, e):
    """Функция реализует метод Зейделя решения СЛАУ.
    Принимает матрицу А левой части и вектор f правой части, 
    необходимую точность e. Возвращает вектор решения"""
    N = A.shape[0]
    diagonal = A.diagonal()
    
    L = np.tril(A, k=-1)
    U = np.triu(A, k=1)
    D = np.diag(np.diag(A))
    
    D_inv = np.linalg.inv(D)
    B = np.eye(N) - D_inv.dot(A)
    norm_B = np.linalg.norm(B, ord=np.inf)
    if norm_B >= 1:
        print("Can't use Zeidel for this system")
        return(-1)
    
    L_D = L + D
    L_D_inv = np.linalg.inv(L_D)
    
    x = []
    x.append(f)
    error = 100
    
    while error > e:
#         print(x[-1])
        x.append(L_D_inv.dot((-U.dot(x[-1]) + f)))
     
        error = np.linalg.norm(x[-1] - x[-2])
        
    print("answer:")
    print(x[-1])
    return(0)
    

In [16]:
print(A_j)
print(f_j)

[[5. 2.]
 [1. 5.]]
[[7.]
 [6.]]


In [17]:
Zeidel(A_j, f_j, 0.001)

answer:
[[0.99991808]
 [1.00001638]]


0

In [18]:
Zeidel(A_test, f_test, 0.001)

Can't use Zeidel for this system


-1

Точное решение (1, 1), что хорошо сходится с полученным по методу Зейделя

## Методы наискорейшего спуска и наименьшей невязки

Для начала, метод нискорейшего спуска

In [19]:
def descent_method(A, f, e):
    """Функция реализует метод наискорейшего спуска решения СЛАУ.
    Принимает матрицу А левой части и вектор f правой части, 
    необходимую точность e. Возвращает вектор решения"""
    x = []
    r = []
    x.append(f)
    error = 100
    
    while error > e:
        r.append(A.dot(x[-1]) - f)
        t = np.transpose(r[-1]).dot(r[-1]) / np.transpose(A.dot(r[-1])).dot(r[-1])
        x.append(x[-1] - t * r[-1])
        error = np.linalg.norm(r[-1])
        
    return x[-1]

In [20]:
print('A:')
print(A_j)
print('f:')
print(f_j)
print('answer:')
descent_method(A_j, f_j, 0.0001)

A:
[[5. 2.]
 [1. 5.]]
f:
[[7.]
 [6.]]
answer:


array([[0.99999934],
       [0.99999945]])

Результат хорошо сходится с точным ответом (1, 1)

Теперь метод наименьших невязок

In [21]:
def minimal_error(A, f, e):
    """Функция реализует метод наименьших невязок решения СЛАУ.
    Принимает матрицу А левой части и вектор f правой части, 
    необходимую точность e. Возвращает вектор решения"""
    x = []
    r = []
    x.append(f)
    error = 100
    
    while error > e:
        r.append(A.dot(x[-1]) - f)
        A_r = A.dot(r[-1])
        t = np.transpose(A_r).dot(r[-1]) / np.transpose(A_r).dot(A_r)
        x.append(x[-1] - t * r[-1])
        error = np.linalg.norm(r[-1])

    return x[-1]

In [22]:
print('A:')
print(A_j)
print('f:')
print(f_j)
print('answer:')
minimal_error(A_j, f_j, 0.001)

A:
[[5. 2.]
 [1. 5.]]
f:
[[7.]
 [6.]]
answer:


array([[0.99998794],
       [1.00002392]])

Результат хорошо сходится с точным ответом (1, 1)

In [23]:
minimal_error(A_d, f_d, 0.001)

array([[1.00112157],
       [1.99958712],
       [2.99921614]])